# Create Shot Corpora

Now that we have ingested our media and decomposed it into shots, we can begin to think about analysis. We won't be wanting to analyse every single frame of the videos, so we need to adopt strategies to best describe our shots with minimal data. We shall perform a number of operations for each shot.

## 1. Setup

First we need to install and import the various libraries we shall be using.

In [1]:
!pip install -q git+https://github.com/jdchart/millefeuille-api.git
!pip install -q git+https://github.com/distant-viewing/dvt.git
!pip install -q git+https://github.com/jdchart/video-analysis.git

import mfapi
import dvt
import jhva
import json
import requests
import os
import cv2
import zipfile
import shutil
from google.colab import files

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

## 2. Gather sources

Next we shall gather all of our sources. We have saved the url's to our shot decomposition Manifests in a json file that we load here.

In [2]:
MANIFEST_PATHS = "https://files.tetras-libre.fr/media/plozevet/fonds_gessain/shot_decomposition_manifests.json"

response = requests.get(MANIFEST_PATHS)
if response.status_code == 200:
    manifets_url_list = json.loads(response.text)["urls"]

print(manifets_url_list)

['https://files.tetras-libre.fr/manifests/plozevet/sources/ebd8f4d7-e88f-4977-be64-d62503eda739.json', 'https://files.tetras-libre.fr/manifests/plozevet/sources/0812d9f7-eb1d-45ae-92f8-5fbe03c1f7fe.json', 'https://files.tetras-libre.fr/manifests/plozevet/sources/243423bb-e6da-4ddc-bf5f-0708c90dab9b.json', 'https://files.tetras-libre.fr/manifests/plozevet/sources/82923da2-73bd-4339-b7c9-451d1a295d7e.json', 'https://files.tetras-libre.fr/manifests/plozevet/sources/84652d38-4e64-406c-a66a-71248d3bf272.json']


We will be needing to download the video for analysis. While we're at it, to get a better idea about what we're dealing with, let's also get some basic data about our sources.

In [3]:
total_video_sources = 0
total_shots = 0
total_frame_count = 0

# Some utility functions:
def dl_video(url, out_path):
  if os.path.isfile(out_path) == False:
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(out_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)

def read_json(url):
  response = requests.get(url)
  if response.status_code == 200:
    return response.json()

if os.path.isdir("sources"):
  shutil.rmtree("sources")
os.makedirs("sources")

for manifest_url in manifets_url_list:
  print(f"Treating {manifest_url}...")

  loaded = read_json(manifest_url)

  total_video_sources = total_video_sources + 1
  total_shots = total_shots + len(loaded["items"][0]["annotations"]["items"])

  dl_path = os.path.join("sources", os.path.splitext(os.path.basename(manifest_url))[0] + ".mp4")
  media_url = loaded["items"][0]["items"][0]["items"][0]["body"]["id"]

  dl_video(media_url, dl_path)

  source_info = dvt.video_info(dl_path)
  total_frame_count = total_frame_count + source_info["frame_count"]

print(f"\nVideo sources: {total_video_sources}")
print(f"Different shots: {total_shots}")
print(f"Total frames: {total_frame_count}")

Treating https://files.tetras-libre.fr/manifests/plozevet/sources/ebd8f4d7-e88f-4977-be64-d62503eda739.json...
Treating https://files.tetras-libre.fr/manifests/plozevet/sources/0812d9f7-eb1d-45ae-92f8-5fbe03c1f7fe.json...
Treating https://files.tetras-libre.fr/manifests/plozevet/sources/243423bb-e6da-4ddc-bf5f-0708c90dab9b.json...
Treating https://files.tetras-libre.fr/manifests/plozevet/sources/82923da2-73bd-4339-b7c9-451d1a295d7e.json...
Treating https://files.tetras-libre.fr/manifests/plozevet/sources/84652d38-4e64-406c-a66a-71248d3bf272.json...

Video sources: 5
Different shots: 2422
Total frames: 291608.0


## 3. Average Pixel Corpus

The first image corpus we're going to create will yield one image for each shot, where the averages of each pixel are taken to create a composite image of the shot. In most cases, the semantic content of the shot is lost, however it will give us a good idea of the colormetric content of the shot.

In [ ]:
OUTPUT_DEST = "frame_averages"

if os.path.isdir(OUTPUT_DEST) == True:
  shutil.rmtree(OUTPUT_DEST)
os.makedirs(OUTPUT_DEST)

for manifest_url in manifets_url_list:
  video_path = os.path.join("sources", os.path.splitext(os.path.basename(manifest_url))[0] + ".mp4")
  annotations = read_json(manifest_url)["items"][0]["annotations"]["items"]

  for i, annotation in enumerate(annotations):
    print(f"Treating shot {i + 1}/{len(annotations)} of {manifest_url}...")

    start, end = annotation["target"].split("#t=")[1].split(",")
    file_name = os.path.splitext(os.path.basename(manifest_url))[0] + f"#shot={i}&time={start},{end}.jpg"
    averager = jhva.FrameAverager(video_path)
    averager.run(os.path.join(OUTPUT_DEST, file_name), start_s = float(start), end_s = float(end))

Don't forget to download the results!

In [ ]:
def zip_folder(folder_path, output_path):
  with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_path):
      for file in files:
        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

zip_folder(OUTPUT_DEST, OUTPUT_DEST + ".zip")
files.download(OUTPUT_DEST + ".zip")

## 4. Keyframe corpus

Another approach is to try and get keyframes for each shot. There are different ways of doing this. Here, we get the pixel data of every n number of frames, use dimensionality reduction to crunch down these numbers, then run a clustering algorithm. An image is taken from each cluster at random.

TODO: make this take the central image of each cluster.

In [ ]:
FRAME_JUMP = 10
NUM_CLUSTERS = 3
OUTPUT_DEST = "keyframes"

if os.path.isdir(OUTPUT_DEST) == True:
  shutil.rmtree(OUTPUT_DEST)
os.makedirs(OUTPUT_DEST)

for manifest_url in manifets_url_list:
  video_path = os.path.join("sources", os.path.splitext(os.path.basename(manifest_url))[0] + ".mp4")
  annotations = read_json(manifest_url)["items"][0]["annotations"]["items"]

  for i, annotation in enumerate(annotations):
    print(f"Treating shot {i + 1}/{len(annotations)} of {manifest_url}...")

    start, end = annotation["target"].split("#t=")[1].split(",")
    file_name = os.path.splitext(os.path.basename(manifest_url))[0] + f"#shot={i}&time={start},{end}"

    keyframer = jhva.Keyframer(video_path, frame_jump = FRAME_JUMP, num_clusters = NUM_CLUSTERS)
    keyframer.run(os.path.join(OUTPUT_DEST, file_name), start_s = float(start), end_s = float(end))

And finally, download the results:

In [6]:
def zip_folder(folder_path, output_path):
  with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_path):
      for file in files:
        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

zip_folder(OUTPUT_DEST, OUTPUT_DEST + ".zip")
files.download(OUTPUT_DEST + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>